In [ ]:
# Name, Category, img_src, Price, Rating, #ofReview, img_name, Link

In [1]:
import os
from bs4 import BeautifulSoup
import re
import html
import requests
import pandas as pd
from time import sleep
from urllib.request import urlretrieve, urlopen
import random
import json
from io import BytesIO
from PIL import Image
from openpyxl import Workbook
from openpyxl import load_workbook

In [11]:
# 성공 : 373649, 381526, 381527, 381528, 381529 ['381523', '381532', '381535', '381538', '381539', '381540', '385001', '385002', '385003', '385004', '385076', '445726', '445727', '445728', '445729', 381523,445730', '445731', '445733', '445860',
# 445861 ~
somelist = ['445861','445863', '445864', '446151', '486319', '486328', '486332', '486342', '486542', '486545', '509024', '509026']

In [4]:
somelist = ['445861']

In [3]:
len(somelist)

12

In [98]:
categoryid

'381523'

In [5]:
baseurl = 'https://www.coupang.com'

if not os.path.exists('./crawling/'):
    os.mkdir('./crawling/')

for b in somelist:
    #여기서 b는 '123456'같은 6자리 카테고리 숫자(but 문자열 취급)
    categoryid = b

    with open('merged/' + categoryid +'.json', 'r') as file:
        loaded_list = json.load(file)

    # 우선 카테고리에 대해, 해당 카테고리 숫자(ex 456788)로 파일 하나 생성
    # 폴더가 존재하는지 확인
    if not os.path.exists('./crawling/'+categoryid):
        print(os.path.exists('./crawling/' + categoryid))

    # 존재하지 않는다면 폴더 생성
        os.mkdir('./crawling/' + categoryid)
        
        
    print(f'{categoryid} 크롤링 시작')
    item_list = []

    for a in loaded_list:
        url = baseurl + a['0']            
            
        resp = requests.get(url,
                           headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
                                          "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3" }
                          )
        soup = BeautifulSoup(resp.text, 'html.parser')
        
        sleep(random.uniform(0.5,1))
            
        try:
            soup.find_all('span', class_='prod-not-find-known__buy__info__txt')[0].get_text() == '현재 판매 중인 상품이 아닙니다.'
            continue

        except:
            pass
        
        try:
            soup.find_all('a', {'class':'prod-not-find-unknown__p'})[0].get_text() == '\n                상품을 찾을 수 없습니다. 주소가 잘못 입력 되었거나, 판매 종료 또는 중지 되어 해당 상품을 찾을 수 없습니다. 이용에 불편을 드려 대단히 죄송합니다.\n            '
            continue
            
        except:
            pass
        
        try:
            soup.find_all('div', {'class':'login__content in-app-login__content--adult'})[0].get_text() == '\n            본 상품은 만 19세 미만의 청소년이 이용할 수 없습니다.\n        '
            continue
            
        except:
            pass

        item_fea = []
        
        
        # Name
        name = soup.find_all('h2', class_='prod-buy-header__title')[0].get_text()        
        
        # Price
        try:
            price =(re.findall('[\d,]+', soup.find_all('span', class_='total-price')[0].get_text())[0])
            
        except IndexError as e:
            continue
        
        # Rating
        rating = float(re.findall(r'"ratingAveragePercentage":[\d.]+', resp.text)[0].split(':')[1])/20
        
        # #ofReview
        review_count = int(re.findall('[\d]+', soup.find_all('span', {'class':'count'})[0].get_text())[0])
        
        # img_src
        img_src = soup.find_all('img', class_='prod-image__detail')[0].get('src')
        
        # img_src 날짜 이후, -로 시작하고 jpg로 끝나는 영문 코드(img_name)
        img_name = re.findall('[\w]+', img_src)[-2]

        # img_name으로 파일 생성
        if not os.path.exists('./crawling/'+categoryid+ '/'+img_name): 
            os.mkdir('./crawling/'+categoryid+ '/'+img_name)

        # img_src을 categoryid 파일 안의 img_name 파일 안에 저장
        response = requests.get('http:' + img_src)
        image = Image.open(BytesIO(response.content))
        image = image.convert('RGB')

        # 이미지 파일로 저장
        image.save('./crawling/'+categoryid+'/'+img_name+'/'+ re.findall('[\w-]+', img_src)[-2] +'.jpg')

        # 그리고 위의 이미지 저장 과정을, 서브 이미지가 있다면 서브 이미지도 해야 함 
        for x in soup.find_all('img', class_='lazy-load-img'):
            x = x.get('data-src')
            x = re.sub('coupangcdn.com/thumbnails/remote/48x48ex/', 'coupangcdn.com/thumbnails/remote/800x800ex/', x)    

            response = requests.get('http:' + x)
            image = Image.open(BytesIO(response.content))
            image = image.convert('RGB')

            # 주소 안의 48x48ex를 사이즈를 수정하면 확대해서 볼 수 있는 것 같은데 수정할까요
            image.save('./crawling/'+categoryid+'/'+img_name+'/'+ re.findall('[\w-]+', x)[-2] +'.jpg')
            
        item_fea.append([name, categoryid, img_src, price, rating, review_count, img_name, url])
        item_list.append(item_fea)
        lst = []
        for i in item_list :
            lst.append(i[0])
        
    # 해당 카테고리 숫자에 대해서 다 크롤링했으면, 엑셀로 저장
    df = pd.DataFrame(lst, columns=['Name', 'Category', 'img_src', 'Price', 'Rating', '#ofReview', 'img_name', 'Link'])
    df.to_excel('./crawling/' +categoryid +'.xlsx', index=False)
    print(f'{categoryid} 크롤링 끝')

False
445861 크롤링 시작
445861 크롤링 끝


In [13]:
url

'https://www.coupang.com/vp/products/6676223662?itemId=15377674375&vendorItemId=82597738061&sourceType=CATEGORY&categoryId=445761'